<a href="https://colab.research.google.com/github/Akshara-P-Vijayan/pdf-rag-chatbot/blob/main/pdf_rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install langchain faiss-cpu transformers pypdf streamlit openai tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [10]:
!pip install -U langchain langchain-community faiss-cpu transformers pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [14]:
!pip install -U langchain-huggingface

In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from transformers import pipeline
import os
from langchain_huggingface import HuggingFacePipeline

In [3]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(documents)

In [4]:
def create_vector_store(chunks):
    embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_documents(chunks, embedder)
    return db


In [5]:
def get_hf_pipeline_llm():
    pipe = pipeline("text-generation", model="tiiuae/falcon-7b-instruct",
                    tokenizer="tiiuae/falcon-7b-instruct",
                    max_new_tokens=200,
                    do_sample=True)
    return HuggingFacePipeline(pipeline=pipe)


In [6]:
def create_qa_chain(db):
    llm = get_hf_pipeline_llm()
    retriever = db.as_retriever(search_type="similarity", k=3)
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    return qa


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
file_path = "/content/drive/MyDrive/pdf-rag-chatbot/src/test1.pdf"
chunks = load_and_split_pdf(file_path)
db = create_vector_store(chunks)
qa_chain = create_qa_chain(db)

query = "What is this PDF about?"
response = qa_chain.invoke(query)

print(response)


/tmp/ipython-input-4-2299747545.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-5-2846859978.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-13-2189972815.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

1.6.3 Depression
of Freezing
Point
Reprint 2025-26

Fig. 1.4: Decrease in the vapour pressure of the
solvent on account of the  presence of
solute in the solvent (a) evaporation of the
molecules of the solvent from its surface
is denoted by , (b) in a solution, solute
particles have been denoted by  and they
also occupy part of the surface area.
Reprint 2025-26

V): The volume percentage is defined as:
Volume % of a component = Volume of  the component 100
Total volume of  solution
(1.2)
1.21.21.2
1.21.2 Expressing ExpressingExpressing ExpressingExpressing
ConcentrationConcentrationConcentrationConcentrationConcentration
of Solutions of Solutionsof Solutions of Solutionsof Solutions
Reprint 2025-26

Kb =
2
1 b
vap
 ×  × 
1000 × 
R M T
H (1.38)
Here the symbols R and M1 stand for the gas constant and molar
mass of the solve